# PyTorch Tutorial: Sequence Models - RNNs to Transformers

From simple RNNs to modern Transformers, this notebook covers the evolution of sequence modeling that powers ChatGPT, Google Translate, and Siri.

## Learning Objectives
- **Part 1**: Understand RNN, LSTM, GRU (The foundations - still asked in interviews!)
- **Part 2**: Implement Seq2Seq models (Translation, summarization)
- **Part 3**: Deep dive into Self-Attention and Transformers
- **Part 4**: Positional Encodings and why they matter
- **Part 5**: GPT vs BERT architectures

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math

torch.manual_seed(42)

---

# PART 2: Transformers and Self-Attention

## From RNN to Transformers: The Paradigm Shift (2017)

**The Problem with RNNs/LSTMs:**
- Sequential processing (can't parallelize)
- Still struggle with very long sequences
- Slow to train

**The Solution: Self-Attention (2017 - "Attention Is All You Need")**
- Process entire sequence in parallel
- Every token attends to every other token directly
- **10x faster to train than LSTMs**

---

## 1. Self-Attention: The Core Mechanism

In a sentence like "The animal didn't cross the street because **it** was too tired", what does "it" refer to? The street or the animal?

Self-attention allows the model to look at ALL other words simultaneously to figure this out.

### The Formula
$$ Attention(Q, K, V) = softmax(\frac{QK^T}{\sqrt{d_k}})V $$

Where:
- **Q (Query)**: What I'm looking for ("Who does 'it' refer to?")
- **K (Key)**: What I have to offer ("I'm the word 'animal'")
- **V (Value)**: What I actually contain (embedding of 'animal')

### The Intuition
1. Each word asks a question (Q): "Which words are relevant to me?"
2. Each word offers information (K): "I'm a noun/verb/etc"
3. Calculate compatibility: Q · K (dot product)
4. Attend to relevant words: softmax(scores) × V

### FAANG Interview Question
**"Implement scaled dot-product attention from scratch"** ← Asked at all FAANG!

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

# Implement attention from scratch for visualization
def scaled_dot_product_attention(Q, K, V, mask=None):
    """
    The EXACT function asked in FAANG interviews!
    
    Args:
        Q: Query matrix (batch, seq_len, d_k)
        K: Key matrix (batch, seq_len, d_k)
        V: Value matrix (batch, seq_len, d_v)
        mask: Optional mask (batch, seq_len, seq_len)
    
    Returns:
        output, attention_weights
    """
    d_k = Q.size(-1)
    
    # Step 1: Compute attention scores (Q @ K^T)
    scores = torch.matmul(Q, K.transpose(-2, -1))  # (batch, seq_len, seq_len)
    
    # Step 2: Scale by sqrt(d_k)
    scores = scores / math.sqrt(d_k)
    
    # Step 3: Apply mask (for causal attention)
    if mask is not None:
        scores = scores.masked_fill(mask == 0, float('-inf'))
    
    # Step 4: Apply softmax
    attention_weights = F.softmax(scores, dim=-1)  # (batch, seq_len, seq_len)
    
    # Step 5: Multiply by V
    output = torch.matmul(attention_weights, V)  # (batch, seq_len, d_v)
    
    return output, attention_weights

# Create a simple example sentence
sentence = "The animal didn't cross the street because it was tired"
tokens = sentence.split()
seq_len = len(tokens)

# Create dummy embeddings (normally from embedding layer)
d_model = 8
torch.manual_seed(42)
embeddings = torch.randn(1, seq_len, d_model)

# Create Q, K, V projections (simplified - normally learned)
W_q = nn.Linear(d_model, d_model, bias=False)
W_k = nn.Linear(d_model, d_model, bias=False)
W_v = nn.Linear(d_model, d_model, bias=False)

Q = W_q(embeddings)
K = W_k(embeddings)
V = W_v(embeddings)

# Compute attention
output, attention_weights = scaled_dot_product_attention(Q, K, V)

# Visualize
fig = plt.figure(figsize=(16, 12))
gs = fig.add_gridspec(3, 3, hspace=0.4, wspace=0.4)

# 1. Attention heatmap
ax1 = fig.add_subplot(gs[0:2, 0:2])
attn_numpy = attention_weights[0].detach().numpy()
sns.heatmap(attn_numpy, annot=True, fmt='.2f', cmap='YlOrRd', 
            xticklabels=tokens, yticklabels=tokens, ax=ax1, 
            cbar_kws={'label': 'Attention Weight'}, vmin=0, vmax=1)
ax1.set_xlabel('Key (What others offer)', fontsize=12)
ax1.set_ylabel('Query (What I\'m looking for)', fontsize=12)
ax1.set_title('Self-Attention Heatmap\n(Row = Query token, Col = Attended token)', 
             fontsize=13, fontweight='bold')
plt.setp(ax1.get_xticklabels(), rotation=45, ha='right')

# Highlight "it" row
it_idx = tokens.index('it')
ax1.add_patch(plt.Rectangle((0, it_idx), len(tokens), 1, 
                            fill=False, edgecolor='blue', linewidth=3))
ax1.text(-1, it_idx + 0.5, '→', fontsize=20, color='blue', 
        ha='right', va='center', fontweight='bold')

# 2. Focus on "it" token
ax2 = fig.add_subplot(gs[0, 2])
it_attention = attn_numpy[it_idx]
colors = ['red' if i == tokens.index('animal') else 'blue' for i in range(seq_len)]
bars = ax2.barh(range(seq_len), it_attention, color=colors, edgecolor='black')
ax2.set_yticks(range(seq_len))
ax2.set_yticklabels(tokens, fontsize=10)
ax2.set_xlabel('Attention Weight', fontsize=11)
ax2.set_title(f'What does "it" attend to?', fontsize=12, fontweight='bold')
ax2.axvline(x=0.1, color='gray', linestyle='--', alpha=0.5)
ax2.grid(True, alpha=0.3, axis='x')

# Add values
for i, (bar, val) in enumerate(zip(bars, it_attention)):
    ax2.text(val + 0.005, i, f'{val:.3f}', va='center', fontsize=9)

# 3. Step-by-step breakdown
ax3 = fig.add_subplot(gs[1, 2])
ax3.axis('off')
steps_text = """
Attention Steps for "it":

1️⃣ Query: What is "it"?
   Q_it = W_q × emb_it
   
2️⃣ Keys: What do others offer?
   K_i = W_k × emb_i for all i
   
3️⃣ Scores: How relevant?
   score_i = Q_it · K_i / √d_k
   
4️⃣ Softmax: Normalize to [0,1]
   attn_i = softmax(scores)
   
5️⃣ Output: Weighted sum
   out_it = Σ attn_i × V_i

Result: "it" attends most to
"animal" (not "street")!
"""
ax3.text(0.05, 0.5, steps_text, fontsize=10, family='monospace',
        bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.8),
        verticalalignment='center')

# 4. QKV matrix visualization
ax4 = fig.add_subplot(gs[2, 0])
ax4.imshow(Q[0, :3, :].detach().numpy(), cmap='Blues', aspect='auto')
ax4.set_yticks(range(3))
ax4.set_yticklabels(tokens[:3])
ax4.set_xlabel('Dimension', fontsize=10)
ax4.set_title('Query Matrix (Q)\n(First 3 tokens)', fontsize=11, fontweight='bold')
ax4.set_xticks([])

ax5 = fig.add_subplot(gs[2, 1])
ax5.imshow(K[0, :3, :].detach().numpy(), cmap='Greens', aspect='auto')
ax5.set_yticks(range(3))
ax5.set_yticklabels(tokens[:3])
ax5.set_xlabel('Dimension', fontsize=10)
ax5.set_title('Key Matrix (K)\n(First 3 tokens)', fontsize=11, fontweight='bold')
ax5.set_xticks([])

ax6 = fig.add_subplot(gs[2, 2])
ax6.imshow(V[0, :3, :].detach().numpy(), cmap='Oranges', aspect='auto')
ax6.set_yticks(range(3))
ax6.set_yticklabels(tokens[:3])
ax6.set_xlabel('Dimension', fontsize=10)
ax6.set_title('Value Matrix (V)\n(First 3 tokens)', fontsize=11, fontweight='bold')
ax6.set_xticks([])

plt.suptitle('Self-Attention Mechanism: Complete Breakdown', 
            fontsize=16, fontweight='bold', y=0.98)
plt.show()

print("🔍 Key Insights:")
print(f"• 'it' attends most to '{tokens[np.argmax(it_attention)]}' (weight: {np.max(it_attention):.3f})")
print(f"• This is learned automatically during training!")
print(f"• Each token can attend to ALL other tokens simultaneously")
print(f"\n💡 FAANG Interview Tip:")
print("Be ready to implement scaled_dot_product_attention() on a whiteboard!")

### Visualization: How Self-Attention Works

Let's visualize the attention mechanism step-by-step with a concrete example.

In [ ]:
class SimpleRNNCell(nn.Module):
    """
    A single RNN cell (one timestep).
    This is the #1 interview question for sequence modeling!
    """
    def __init__(self, input_size, hidden_size):
        super().__init__()
        self.hidden_size = hidden_size
        
        # Weights for input -> hidden
        self.W_xh = nn.Linear(input_size, hidden_size)
        # Weights for hidden -> hidden (the recurrent part!)
        self.W_hh = nn.Linear(hidden_size, hidden_size, bias=False)
        
        self.tanh = nn.Tanh()
    
    def forward(self, x, h_prev):
        """
        x: input at current timestep (batch_size, input_size)
        h_prev: hidden state from previous timestep (batch_size, hidden_size)
        """
        h_t = self.tanh(self.W_xh(x) + self.W_hh(h_prev))
        return h_t

# Test the RNN cell
input_size = 10
hidden_size = 20
batch_size = 4

rnn_cell = SimpleRNNCell(input_size, hidden_size)

# Initialize hidden state
h = torch.zeros(batch_size, hidden_size)

# Process a sequence of 5 timesteps
sequence_length = 5
for t in range(sequence_length):
    x_t = torch.randn(batch_size, input_size)
    h = rnn_cell(x_t, h)
    print(f"Timestep {t}: hidden state shape = {h.shape}")

print("\\n✓ RNN processes sequence step-by-step, maintaining hidden state!")

---

## 2. Positional Encodings: Why and How

### The Problem
Self-attention has NO concept of order!
- "Dog bites man" and "Man bites dog" look identical to attention.
- RNNs have built-in order (process sequentially), but Transformers don't.

### The Solution: Positional Encodings
Add position information to each token embedding.

$$
\begin{align*}
PE_{(pos, 2i)} &= \sin(pos / 10000^{2i/d_{model}}) \\
PE_{(pos, 2i+1)} &= \cos(pos / 10000^{2i/d_{model}})
\end{align*}
$$

Where:
- $pos$: Position in sequence (0, 1, 2, ...)
- $i$: Dimension index
- $d_{model}$: Embedding dimension

### Why Sinusoidal?
1. **Unique** for each position
2. **Relative** positions have consistent patterns
3. **Generalizes** to longer sequences than seen in training

### Modern Alternatives (2024+)
- **Learned PE**: Train positional embeddings like word embeddings
- **RoPE** (Rotary): Used in Llama, PaLM (multiplies instead of adds)
- **ALiBi**: Attention bias based on distance

### FAANG Interview Question
**"Why do Transformers need positional encodings?"** ← Asked at Google, OpenAI

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

class PositionalEncoding(nn.Module):
    """
    Sinusoidal positional encoding (from "Attention Is All You Need").
    This is THE standard implementation asked in interviews!
    """
    def __init__(self, d_model, max_len=5000):
        super().__init__()
        
        # Create matrix of shape (max_len, d_model)
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        
        # div_term = 10000^(2i/d_model)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * 
                            (-math.log(10000.0) / d_model))
        
        # Apply sin to even indices, cos to odd indices
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        
        # Add batch dimension: (1, max_len, d_model)
        pe = pe.unsqueeze(0)
        
        # Register as buffer (not a parameter, but saved with model)
        self.register_buffer('pe', pe)
    
    def forward(self, x):
        """
        x: (batch, seq_len, d_model)
        """
        # Add positional encoding to input
        x = x + self.pe[:, :x.size(1), :]
        return x

# Visualize positional encodings
d_model = 128
max_len = 100

pos_enc = PositionalEncoding(d_model, max_len)
pe_matrix = pos_enc.pe.squeeze(0).numpy()

# Plot
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.imshow(pe_matrix.T, aspect='auto', cmap='RdBu', interpolation='nearest')
plt.xlabel('Position')
plt.ylabel('Dimension')
plt.title('Positional Encoding Matrix')
plt.colorbar()

plt.subplot(1, 2, 2)
plt.plot(pe_matrix[:, :4])
plt.xlabel('Position')
plt.ylabel('Value')
plt.title('First 4 Dimensions')
plt.legend([f'Dim {i}' for i in range(4)])
plt.grid(True)

plt.tight_layout()
plt.show()

print("✓ Each position gets a unique encoding!")
print("✓ Sinusoidal pattern allows model to learn relative positions.")

## The Problem with RNNs: Vanishing Gradients

When sequences get long (100+ tokens), gradients vanish during backprop:
- Gradient flows: Output → h_100 → h_99 → ... → h_1
- Each step multiplies by a matrix
- Repeated multiplication makes gradients explode or vanish

**Result:** RNNs can't remember long-term dependencies.

---

## LSTM: Long Short-Term Memory (1997, Still Used Today!)

### The Breakthrough
Add explicit **memory cells** with **gates** that control:
1. **Forget gate**: What to remove from memory
2. **Input gate**: What new information to add
3. **Output gate**: What to expose as hidden state

### The Equations (Don't Panic!)
$$
\begin{align*}
f_t &= \sigma(W_f \cdot [h_{t-1}, x_t] + b_f) \quad &\text{(Forget gate)} \\
i_t &= \sigma(W_i \cdot [h_{t-1}, x_t] + b_i) \quad &\text{(Input gate)} \\
\tilde{C}_t &= \tanh(W_C \cdot [h_{t-1}, x_t] + b_C) \quad &\text{(Candidate values)} \\
C_t &= f_t * C_{t-1} + i_t * \tilde{C}_t \quad &\text{(Update cell state)} \\
o_t &= \sigma(W_o \cdot [h_{t-1}, x_t] + b_o) \quad &\text{(Output gate)} \\
h_t &= o_t * \tanh(C_t) \quad &\text{(Final hidden state)}
\end{align*}
$$

### The Intuition
- **Cell state** ($C_t$): The "highway" where information flows unmodified
- **Gates**: Traffic lights that decide what flows through

### FAANG Interview Alert
**"Explain the difference between RNN and LSTM"** ← Top 5 most asked question!

In [ ]:
class LSTMCell(nn.Module):
    """
    LSTM Cell from scratch - asked at Google, Meta, Amazon!
    """
    def __init__(self, input_size, hidden_size):
        super().__init__()
        self.hidden_size = hidden_size
        
        # All 4 gates can be computed together (efficiency trick!)
        # Input: [h_{t-1}; x_t] concatenated
        self.gates = nn.Linear(input_size + hidden_size, 4 * hidden_size)
    
    def forward(self, x, states):
        """
        x: (batch, input_size)
        states: (h_prev, c_prev) where each is (batch, hidden_size)
        """
        h_prev, c_prev = states
        
        # Concatenate input and hidden state
        combined = torch.cat([x, h_prev], dim=1)
        
        # Compute all gates at once, then split
        gates = self.gates(combined)
        i, f, g, o = gates.chunk(4, dim=1)
        
        # Apply activations
        i = torch.sigmoid(i)  # Input gate
        f = torch.sigmoid(f)  # Forget gate
        g = torch.tanh(g)     # Candidate values
        o = torch.sigmoid(o)  # Output gate
        
        # Update cell state (the memory!)
        c_t = f * c_prev + i * g
        
        # Update hidden state
        h_t = o * torch.tanh(c_t)
        
        return h_t, c_t

# Test LSTM
lstm_cell = LSTMCell(input_size=10, hidden_size=20)

h = torch.zeros(4, 20)
c = torch.zeros(4, 20)

print("LSTM processing sequence:")
for t in range(5):
    x_t = torch.randn(4, 10)
    h, c = lstm_cell(x_t, (h, c))
    print(f"Step {t}: h={h.shape}, c={c.shape}")

print("\\n✓ LSTM maintains TWO states: hidden (h) and cell (c)!")
print("The cell state (c) is the 'memory highway' that solves vanishing gradients.")

---

## Seq2Seq: Sequence-to-Sequence Models (The Bridge to Transformers)

### The Problem
- RNN/LSTM can process one sequence → one output
- But what about sequence → sequence? (Translation, Summarization)
  - Input: "Hello world" → Output: "Hola mundo"

### The Solution: Encoder-Decoder Architecture (2014)

```
Encoder (LSTM)                Decoder (LSTM)
    ↓                              ↓
Input Sequence  →  Context Vector  →  Output Sequence
"Hello world"       (fixed size)      "Hola mundo"
```

**How it works:**
1. **Encoder**: Reads input sequence, compresses to fixed-size vector (context)
2. **Decoder**: Generates output sequence from context vector

### The Problem with Seq2Seq
**Bottleneck**: All information compressed into ONE vector!
- Long sequences lose information
- This is what **Attention** was invented to solve (2015)

### FAANG Interview Question
**"Explain the Seq2Seq architecture and its limitation"** ← Asked at Google, Meta

**Answer:**
1. Encoder processes input → context vector
2. Decoder generates output from context
3. **Limitation**: Fixed-size context is a bottleneck for long sequences
4. **Solution**: Attention mechanism (attend to different encoder states)

In [ ]:
# Seq2Seq Implementation (Simplified)

class Encoder(nn.Module):
    """Encodes input sequence into a context vector"""
    def __init__(self, input_size, hidden_size):
        super().__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size, batch_first=True)
    
    def forward(self, x):
        # x: (batch, seq_len) - token IDs
        embedded = self.embedding(x)  # (batch, seq_len, hidden_size)
        outputs, (hidden, cell) = self.lstm(embedded)
        # Return final hidden state as context
        return hidden, cell

class Decoder(nn.Module):
    """Generates output sequence from context vector"""
    def __init__(self, output_size, hidden_size):
        super().__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x, hidden, cell):
        # x: (batch, 1) - one token at a time
        embedded = self.embedding(x)  # (batch, 1, hidden_size)
        output, (hidden, cell) = self.lstm(embedded, (hidden, cell))
        prediction = self.fc(output.squeeze(1))  # (batch, output_size)
        return prediction, hidden, cell

class Seq2Seq(nn.Module):
    """
    Complete Seq2Seq model for sequence-to-sequence tasks.
    Used in early machine translation systems (pre-Transformer).
    """
    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
    
    def forward(self, src, trg, teacher_forcing_ratio=0.5):
        """
        src: source sequence (batch, src_len)
        trg: target sequence (batch, trg_len)
        teacher_forcing_ratio: probability of using true token vs predicted
        """
        batch_size = src.size(0)
        trg_len = trg.size(1)
        trg_vocab_size = self.decoder.fc.out_features
        
        # Store outputs
        outputs = torch.zeros(batch_size, trg_len, trg_vocab_size)
        
        # Encode entire input sequence
        hidden, cell = self.encoder(src)
        
        # First input to decoder is <SOS> token
        input_token = trg[:, 0].unsqueeze(1)
        
        # Generate output sequence one token at a time
        for t in range(1, trg_len):
            output, hidden, cell = self.decoder(input_token, hidden, cell)
            outputs[:, t, :] = output
            
            # Teacher forcing: use true token or predicted token?
            use_teacher_forcing = torch.rand(1).item() < teacher_forcing_ratio
            top1 = output.argmax(1).unsqueeze(1)
            input_token = trg[:, t].unsqueeze(1) if use_teacher_forcing else top1
        
        return outputs

# Example
vocab_size = 1000
hidden_size = 256

encoder = Encoder(vocab_size, hidden_size)
decoder = Decoder(vocab_size, hidden_size)
seq2seq = Seq2Seq(encoder, decoder)

# Dummy data
src = torch.randint(0, vocab_size, (4, 10))  # Batch of 4, length 10
trg = torch.randint(0, vocab_size, (4, 12))  # Batch of 4, length 12

outputs = seq2seq(src, trg)
print(f"Input shape: {src.shape}")
print(f"Target shape: {trg.shape}")
print(f"Output shape: {outputs.shape}")
print("\\n✓ Seq2Seq translates variable-length sequences!")
print("\\nThis was state-of-the-art for translation (2014-2017)")
print("Then Transformers came and changed everything...")

In [ ]:
# Visualize Multi-Head Attention and Causal Masking
fig = plt.figure(figsize=(18, 10))
gs = fig.add_gridspec(2, 3, hspace=0.35, wspace=0.35)

# Simpler sentence for clearer visualization
sentence2 = "I love deep learning"
tokens2 = sentence2.split()
seq_len2 = len(tokens2)

torch.manual_seed(42)
embeddings2 = torch.randn(1, seq_len2, 16)

# 1. Multi-Head Attention - Different heads learn different patterns
ax1 = fig.add_subplot(gs[0, 0])
ax1.axis('off')
ax1.text(0.5, 0.9, 'Multi-Head Attention', ha='center', fontsize=14, 
        fontweight='bold', transform=ax1.transAxes)

# Simulate 3 different attention heads
num_heads = 3
head_names = ['Head 1:\nSyntax', 'Head 2:\nSemantics', 'Head 3:\nContext']

for head_idx in range(num_heads):
    # Create different attention patterns for each head
    torch.manual_seed(40 + head_idx)
    W_q_head = nn.Linear(16, 8, bias=False)
    W_k_head = nn.Linear(16, 8, bias=False)
    W_v_head = nn.Linear(16, 8, bias=False)
    
    Q_head = W_q_head(embeddings2)
    K_head = W_k_head(embeddings2)
    V_head = W_v_head(embeddings2)
    
    _, attn_head = scaled_dot_product_attention(Q_head, K_head, V_head)
    
    # Create subplot for this head
    left = 0.05 + head_idx * 0.3
    ax_head = fig.add_axes([left, 0.55, 0.22, 0.25])
    
    sns.heatmap(attn_head[0].detach().numpy(), annot=True, fmt='.2f', 
                cmap='YlOrRd', xticklabels=tokens2, yticklabels=tokens2,
                ax=ax_head, cbar=False, vmin=0, vmax=1, linewidths=0.5)
    ax_head.set_title(head_names[head_idx], fontsize=11, fontweight='bold')
    ax_head.set_xlabel('', fontsize=9)
    ax_head.set_ylabel('', fontsize=9)
    plt.setp(ax_head.get_xticklabels(), rotation=45, ha='right', fontsize=9)
    plt.setp(ax_head.get_yticklabels(), fontsize=9)

# 2. BERT: Bidirectional Attention (no mask)
ax2 = fig.add_subplot(gs[1, 0])
torch.manual_seed(42)
W_q2 = nn.Linear(16, 16, bias=False)
W_k2 = nn.Linear(16, 16, bias=False)
W_v2 = nn.Linear(16, 16, bias=False)

Q2 = W_q2(embeddings2)
K2 = W_k2(embeddings2)
V2 = W_v2(embeddings2)

_, attn_bidirectional = scaled_dot_product_attention(Q2, K2, V2, mask=None)
sns.heatmap(attn_bidirectional[0].detach().numpy(), annot=True, fmt='.2f',
            cmap='Greens', xticklabels=tokens2, yticklabels=tokens2,
            ax=ax2, cbar_kws={'label': 'Weight'}, vmin=0, vmax=1)
ax2.set_title('BERT: Bidirectional Attention\n(Can see ALL tokens)', 
             fontsize=12, fontweight='bold', color='darkgreen')
ax2.set_xlabel('Key', fontsize=10)
ax2.set_ylabel('Query', fontsize=10)
plt.setp(ax2.get_xticklabels(), rotation=45, ha='right')

# 3. GPT: Causal Attention (with mask)
ax3 = fig.add_subplot(gs[1, 1])

# Create causal mask (lower triangular)
causal_mask = torch.tril(torch.ones(seq_len2, seq_len2))
_, attn_causal = scaled_dot_product_attention(Q2, K2, V2, mask=causal_mask)

sns.heatmap(attn_causal[0].detach().numpy(), annot=True, fmt='.2f',
            cmap='Blues', xticklabels=tokens2, yticklabels=tokens2,
            ax=ax3, cbar_kws={'label': 'Weight'}, vmin=0, vmax=1, mask=(causal_mask == 0).numpy())
ax3.set_title('GPT: Causal Attention\n(Can only see PAST tokens)', 
             fontsize=12, fontweight='bold', color='darkblue')
ax3.set_xlabel('Key', fontsize=10)
ax3.set_ylabel('Query', fontsize=10)
plt.setp(ax3.get_xticklabels(), rotation=45, ha='right')

# Add gray for masked regions
for i in range(seq_len2):
    for j in range(i+1, seq_len2):
        ax3.add_patch(plt.Rectangle((j, i), 1, 1, fill=True, 
                                    facecolor='gray', alpha=0.7, edgecolor='white'))
        ax3.text(j+0.5, i+0.5, 'X', ha='center', va='center', 
                color='white', fontweight='bold', fontsize=12)

# 4. Comparison explanation
ax4 = fig.add_subplot(gs[1, 2])
ax4.axis('off')

comparison_text = """
BERT vs GPT Attention:

━━━━━━━━━━━━━━━━━━━━━━━━━

BERT (Encoder):
✓ Bidirectional
✓ Sees entire sentence
✓ Best for: Classification,
  NER, Q&A, embeddings
  
Example:
"I love [MASK] learning"
→ Can see both sides!

━━━━━━━━━━━━━━━━━━━━━━━━━

GPT (Decoder):
✓ Causal (autoregressive)
✓ Only sees past tokens
✓ Best for: Generation,
  completion, chat
  
Example:
"I love deep" → predict next
→ Cannot see "learning"!

━━━━━━━━━━━━━━━━━━━━━━━━━

The Mask:
• Gray X = -∞ before softmax
• Prevents "cheating"
• Critical for generation!

━━━━━━━━━━━━━━━━━━━━━━━━━

FAANG Interview:
"Implement causal mask"
→ torch.tril() creates
   lower-triangular matrix
"""

ax4.text(0.05, 0.5, comparison_text, fontsize=9.5, family='monospace',
        bbox=dict(boxstyle='round', facecolor='lightyellow', alpha=0.9,
                 edgecolor='black', linewidth=2),
        verticalalignment='center')

plt.suptitle('Multi-Head Attention & Causal Masking (BERT vs GPT)', 
            fontsize=16, fontweight='bold')
plt.show()

print("🎯 Key Takeaways for FAANG Interviews:")
print("\n1. Multi-Head Attention:")
print("   • Run attention multiple times in parallel")
print("   • Each head learns different patterns (syntax, semantics, etc.)")
print("   • Concat all heads then project: Concat(head1, head2, ...) @ W_o")

print("\n2. BERT vs GPT:")
print("   • BERT: Bidirectional (no mask) → Understanding tasks")
print("   • GPT: Causal (with mask) → Generation tasks")
print("   • Mask = torch.tril() creates lower triangular matrix")

print("\n3. Why Causal Masking?")
print("   • Prevents the model from 'cheating' during generation")
print("   • At inference, we generate token-by-token (can't see future)")
print("   • Training must match inference conditions!")

print("\n💡 Interview Question:")
print('Q: "Why does GPT use causal masking?"')
print('A: "To match training and inference. During generation, we predict')
print('    one token at a time without seeing future tokens, so training')
print('    must enforce the same constraint."')

### Visualization: Multi-Head Attention & Causal Masking

Let's visualize how multi-head attention works and the critical difference between BERT and GPT.

## 1. The Core Idea: Self-Attention

In a sentence like "The animal didn't cross the street because **it** was too tired", what does "it" refer to? The street or the animal?

Self-attention allows the model to look at other words in the sentence to figure this out. It computes a weighted sum of all other words.

Formula:
$$ Attention(Q, K, V) = softmax(\frac{QK^T}{\sqrt{d_k}})V $$

Where:
- **Q (Query)**: What I'm looking for
- **K (Key)**: What I have to offer
- **V (Value)**: What I actually contain

In [ ]:
---

# PART 3: GPT vs BERT Architectures (Must Know!)

This is **THE most important interview topic for NLP roles in 2025**.

## The Two Paradigms

### BERT: Bidirectional Encoder (Google, 2018)
- **Architecture**: Encoder-only
- **Training**: Masked Language Modeling (predict masked words)
- **Use case**: Understanding (classification, NER, Q&A)
- **Example**: "The [MASK] is shining" → predict "sun"
- **Key**: Sees ENTIRE sentence (bidirectional context)

### GPT: Autoregressive Decoder (OpenAI, 2018)
- **Architecture**: Decoder-only
- **Training**: Next token prediction (left-to-right)
- **Use case**: Generation (text completion, chat)
- **Example**: "The sun is" → predict "shining"
- **Key**: Sees ONLY previous tokens (causal/autoregressive)

---

## The Critical Difference: Attention Masking

### BERT (Bidirectional)
```
Attention weights for "is":
  The  sun  is shining
  ✓    ✓   ✓    ✓      (can attend to ALL words)
```

### GPT (Causal)  
```
Attention weights for "is":
  The  sun  is shining
  ✓    ✓   ✓    ✗      (can ONLY attend to past, not future!)
```

---

## FAANG Interview Questions

**Q1: "When would you use BERT vs GPT?"**
```
BERT:
- Classification (sentiment, spam detection)
- Named Entity Recognition
- Question Answering
- Sentence embeddings

GPT:
- Text generation
- Chatbots
- Code completion
- Creative writing
```

**Q2: "How do you implement causal masking?"**
```python
# This creates a lower-triangular matrix
# Future tokens are masked with -inf
mask = torch.tril(torch.ones(seq_len, seq_len))
mask = mask.masked_fill(mask == 0, float('-inf'))
scores = scores + mask  # Before softmax
```

**Q3: "Why can't BERT generate text?"**
```
Answer: BERT is trained to predict masked words using 
bidirectional context. It's not trained for autoregressive 
generation (predicting next token). You'd need to fine-tune 
it differently or use a decoder.
```

---

## Architecture Comparison Table

| Feature | BERT | GPT |
|---------|------|-----|
| **Type** | Encoder | Decoder |
| **Attention** | Bidirectional | Causal (masked) |
| **Training** | MLM (predict masks) | Next token prediction |
| **Input** | [CLS] text [SEP] | text |
| **Output** | Token embeddings | Next token logits |
| **Best for** | Understanding | Generation |
| **Examples** | RoBERTa, ALBERT | GPT-3, ChatGPT |

---

## What Powers ChatGPT?
- **Architecture**: Decoder-only (GPT)
- **Size**: 175B parameters (GPT-3) → 1.76T parameters (GPT-4 rumored)
- **Training**: Causal language modeling + RLHF
- **Context**: 8K tokens (GPT-3) → 128K tokens (GPT-4 Turbo)

---

## Key Takeaways

1. **BERT = Encoder = Understanding**
2. **GPT = Decoder = Generation**
3. **Causal mask** is the key difference
4. **Modern trend**: Decoder-only models (Llama, PaLM, GPT) dominate
5. **Why?**: Decoders can do both understanding AND generation

## 2. PyTorch's Transformer Modules

PyTorch provides optimized implementations so you don't have to write everything from scratch.

In [ ]:
# Single Multi-Head Attention Layer
multihead_attn = nn.MultiheadAttention(embed_dim=256, num_heads=8, batch_first=True)

# Create dummy input: (Batch, Seq_Len, Embed_Dim)
x = torch.randn(32, 10, 256)

# Self-attention: Q=x, K=x, V=x
attn_output, _ = multihead_attn(x, x, x)
print(f"Input shape: {x.shape}")
print(f"Output shape: {attn_output.shape}")

---

# PART 4: Tokenization Deep Dive (Critical for LLMs)

## The Problem
Neural networks work with numbers, not text. How do we convert text → numbers?

### Naive Approaches (Don't Use!)
1. **Character-level**: "Hello" → [H, e, l, l, o] → Too long, loses word meaning
2. **Word-level**: "Hello" → [Hello] → Huge vocabulary (1M+ words)

### Modern Solutions: Subword Tokenization

## BPE (Byte Pair Encoding) - Used in GPT

**Idea:** Start with characters, merge most frequent pairs

```
Step 1: "lower" → ["l", "o", "w", "e", "r"]
Step 2: Merge "e"+"r" → ["l", "o", "w", "er"]  (if most frequent)
Step 3: Merge "l"+"o" → ["lo", "w", "er"]
...
```

**Result:** 
- Common words: single token ("the" → [the])
- Rare words: multiple subtokens ("chatbot" → ["chat", "bot"])
- Unknown words: character-level fallback

## WordPiece - Used in BERT

Similar to BPE but uses likelihood-based merging.

## SentencePiece - Used in Llama, T5

Language-agnostic, works directly on Unicode.

---

## FAANG Interview Questions

**Q1: "Why use subword tokenization instead of word-level?"**
```
Advantages:
1. Fixed vocabulary size (32K-100K vs 1M+ words)
2. Handles rare/unknown words (no <UNK> token)
3. Captures morphology ("play", "playing", "played" share "play")
4. Language-agnostic
```

**Q2: "What's the difference between BPE and WordPiece?"**
```
BPE: Merge most frequent pairs
WordPiece: Merge pairs that maximize likelihood on training data
Both achieve similar results in practice.
```

**Q3: "How does GPT tokenize '🤖'?"**
```
Emojis, special characters: Usually single or multi-byte tokens
GPT uses UTF-8 bytes → BPE
Most modern tokenizers handle Unicode natively
```

---

## Tokenization in Practice

```python
from transformers import AutoTokenizer

# Load GPT-2 tokenizer (BPE)
tokenizer = AutoTokenizer.from_pretrained("gpt2")

text = "Hello, world!"
tokens = tokenizer.tokenize(text)
ids = tokenizer.encode(text)

print(f"Text: {text}")
print(f"Tokens: {tokens}")
print(f"IDs: {ids}")

# Decode back
decoded = tokenizer.decode(ids)
print(f"Decoded: {decoded}")
```

---

## Key Concepts

### Special Tokens
- `[CLS]`: Classification token (BERT)
- `[SEP]`: Separator (BERT)
- `<BOS>`: Beginning of sequence (GPT)
- `<EOS>`: End of sequence
- `<PAD>`: Padding (for batching)
- `<UNK>`: Unknown word (rare in BPE)

### Vocabulary Size Impact
- **Small vocab (8K)**: More tokens per sentence, longer sequences
- **Large vocab (100K)**: Fewer tokens per sentence, but larger embedding matrix
- **Sweet spot**: 32K-50K for most LLMs

---

## Tokenization Challenges

1. **Numbers**: "1234" might be ["1", "23", "4"] - loses semantic meaning
2. **Code**: Indentation, brackets often split poorly
3. **Multilingual**: English-centric tokenizers waste tokens on other languages
4. **Trailing spaces**: "Hello" vs "Hello " are different tokens!

---

## Modern Trends (2024-2025)

- **Larger vocabs**: GPT-4 likely uses 100K+ (unconfirmed)
- **Multimodal**: Image patches as "visual tokens" (CLIP, Flamingo)
- **Byte-level**: Direct UTF-8 bytes (more universal)

---

**Tokenization is the "dark art" of NLP - small changes have huge impacts!**

In [ ]:
encoder_layer = nn.TransformerEncoderLayer(d_model=256, nhead=8, batch_first=True)
transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=6)

output = transformer_encoder(x)
print(f"Encoder Output shape: {output.shape}")

## 4. Tokenization (Concept)

Transformers don't understand text; they understand numbers. Tokenization converts text to numbers.

1. Text: "I love AI"
2. Tokens: ["I", "love", "AI"]
3. IDs: [101, 204, 505]

*(In practice, we use libraries like `huggingface/tokenizers`)*

## Key Takeaways

1. **Self-Attention**: The mechanism that relates different positions of a sequence.
2. **Q, K, V**: The three projections used to compute attention.
3. **Multi-Head Attention**: Running multiple attention mechanisms in parallel.
4. **Transformer**: A stack of attention and feed-forward layers.